# Stanford CS25 Review: Transformers in Biomedicine
### From General-Purpose Giants to Specialized Experts

Welcome to this deep dive into the exciting intersection of AI and biomedicine, inspired by the Stanford CS25 lecture on Biomedical Transformers. This notebook serves as a self-contained guide, breaking down the core concepts from the lecture and the pivotal research paper, **"Do We Still Need Clinical Language Models?"**. Our goal is to make these advanced topics accessible, intuitive, and memorable, so you won't need to watch the hour-long video to grasp the key takeaways.

We'll explore the central debate in medical AI today:

1.  **The Generalists (e.g., Med-PaLM):** Can massive, general-purpose Large Language Models (LLMs), trained on the entire web, be adapted to master the complexities of medicine?
2.  **The Specialists (e.g., Clinical-T5):** Is there still a fundamental need for smaller, more efficient models that are meticulously trained *specifically* on clinical data?

Let's dive in!

## Why Transformers are a Natural Fit for Biomedicine

At its core, much of biomedical data is made of **sequences**. The lecture beautifully illustrates this:

-   **Clinical Notes & Electronic Health Records (EHR):** A sequence of a patient's interactions, diagnoses, and treatments over time.
-   **Proteins:** A sequence of amino acids.
-   **Genomes:** A long, long sequence of nucleotide base pairs (A, C, G, T).

Transformers are the state-of-the-art architecture for modeling sequences for several key reasons:

| Feature | Why it Matters in Biomedicine | Analogy |
| :--- | :--- | :--- |
| **Modeling Long-Range Interactions** | A genetic mutation far away on a chromosome can affect a gene's function. An event in a patient's childhood can be relevant to a diagnosis decades later. Transformers can connect these distant dots. | A skilled detective who remembers a clue from the first chapter to solve the mystery in the last. |
| **Handling Multimodality** | Transformers can process diverse data types—text from notes, structured data from lab results, and even sequences from genomics—within a unified framework. | A universal translator that can fluently switch between speaking, writing, and sign language. |
| **Scalability & Parallelization** | Biomedical datasets are enormous (billions of base pairs in a genome!). Transformers can be trained efficiently on massive datasets using modern hardware (GPUs/TPUs). | A modern factory assembly line that can produce goods at a massive scale, far surpassing a single artisan. |

--- 
## Part 1: The Generalists - The Rise of Med-PaLM

The first major part of the lecture focuses on the paper **"Large Language Models Encode Clinical Knowledge,"** which introduced **Med-PaLM**. This work represents the "generalist" approach: taking a massive, web-trained model and teaching it to be a medical expert.

### The Challenge: Evaluating Medical AI is Hard

Before Med-PaLM, a major hurdle was the lack of a comprehensive benchmark to evaluate medical AI. Existing tests were narrow. To solve this, the researchers created two things:

1.  **MultiMedQA Benchmark:** A collection of seven diverse medical question-answering datasets, unifying everything from professional medical exam questions to casual consumer health queries.
2.  **Human Evaluation Framework:** A rigorous system where both expert clinicians and everyday lay users rated model answers on multiple axes, including factuality, reasoning, potential for harm, and helpfulness. This goes far beyond simple accuracy scores.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data representing the conceptual composition of the MultiMedQA benchmark
datasets = ['USMLE-Style (MedQA)', 'Medical Research (PubMedQA)', 'Consumer Questions (LiveQA)', 
            'New Consumer Qs (HealthSearchQA)', 'Indian Exams (MedMCQA)', 'Clinical Topics (MMLU)']
difficulty = [10, 8, 4, 5, 9, 7] # Arbitrary difficulty/specialization score
colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4', '#9467bd', '#8c564b']

plt.figure(figsize=(10, 7))
plt.barh(datasets, difficulty, color=colors, alpha=0.8)
plt.xlabel('Conceptual Difficulty & Specialization', fontsize=12)
plt.title('Conceptual Overview of the MultiMedQA Benchmark', fontsize=16, pad=20)
plt.gca().invert_yaxis() # To have the most difficult at the top
plt.grid(axis='x', linestyle='--', alpha=0.6)

plt.text(0, -0.8, 'MultiMedQA combines datasets of varying types and complexities to create a robust evaluation suite.', 
         ha='left', fontsize=12, wrap=True)

plt.tight_layout()
plt.show()

### The Solution: From Raw Power (FLAN-PaLM) to Refined Skill (Med-PaLM)

The researchers started with **FLAN-PaLM**, a 540-billion parameter general-purpose LLM. While it excelled at multiple-choice questions (even passing the US Medical Licensing Exam), its long-form answers often fell short. They were sometimes incomplete, factually incorrect (a phenomenon known as "hallucination"), and consistently rated as inferior to answers written by human clinicians.

The model had immense knowledge but lacked the specialized skill to apply it safely and effectively in a medical context. It was like a brilliant library that didn't know how to organize or present its information.

The solution was a clever and efficient technique called **Instruction Prompt Tuning**.

#### Analogy: Training a Brilliant but Unfocused Student

Imagine you have a genius student (the LLM) who has read every book in the world but struggles with exams. 

**Full Finetuning (The Hard Way):** You could try to re-teach the student every subject from scratch. This is slow, expensive, and risks confusing them.
**Instruction Prompt Tuning (The Smart Way):** Instead, you freeze their existing knowledge and just teach them *how to take the exam*. You give them a few hundred examples of perfect answers, showing them the right tone, structure, and level of detail. The student learns a small set of "instructions" (the prompt vectors) on how to access and present their vast knowledge for this specific task. 

This is exactly what the researchers did. They used a small set of high-quality examples from expert clinicians to tune the model, creating **Med-PaLM**.

In [ ]:
# This code simulates and visualizes the performance jump from FLAN-PaLM to Med-PaLM

labels = ['Agrees with Scientific Consensus', 'Avoids Incorrect Content',
          'Avoids Harmful Content', 'Helpful (Lay User Rating)']

# Simulated data based on trends described in the lecture
flan_palm_scores = np.array([62, 58, 94.2, 61])
med_palm_scores = np.array([92.6, 93.1, 99.9, 80.3])
clinician_scores = np.array([92.9, 94.0, 100.0, 91.1])

x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(14, 8))
rects1 = ax.bar(x - width, flan_palm_scores, width, label='FLAN-PaLM (Generalist)', color='#ff9896')
rects2 = ax.bar(x, med_palm_scores, width, label='Med-PaLM (Tuned)', color='#aec7e8')
rects3 = ax.bar(x + width, clinician_scores, width, label='Clinician Answer', color='#98df8a')

ax.set_ylabel('Performance / Agreement (%)', fontsize=12)
ax.set_title('Med-PaLM Closes the Gap to Expert Clinicians', fontsize=16, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=15, ha='right')
ax.set_ylim(50, 105)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)

ax.bar_label(rects1, padding=3, fmt='%.1f')
ax.bar_label(rects2, padding=3, fmt='%.1f')
ax.bar_label(rects3, padding=3, fmt='%.1f')

fig.tight_layout()
plt.show()

The chart clearly shows that while the generalist `FLAN-PaLM` was decent, the `Med-PaLM` model, after instruction tuning, dramatically improved across the board, becoming nearly indistinguishable from (and in some ways, complementary to) expert human clinicians.

---
## Part 2: The Specialists - The Case for Clinical Language Models

While Med-PaLM is impressive, it's a massive, resource-intensive model. This brings us to the core question of the second paper discussed: **"Do We Still Need Clinical Language Models?"**

This research investigates whether smaller models, trained specifically on mountains of messy, jargon-filled *clinical notes* (like those in the MIMIC dataset), can outperform the big generalists on certain tasks.

### The Four Philosophies of Model Training

The paper frames the problem by comparing four distinct approaches to solving a clinical task. Let's use an analogy of training a chef for a highly specialized cuisine, like molecular gastronomy.

| Approach | Model Strategy | Chef Analogy |
|:---|:---|:---|
| **1. Train from Scratch (Specialist)** | Train a model only on clinical text. | Train a chef from day one *only* in molecular gastronomy. They become a pure expert. |
| **2. Domain-Adaptive Pretraining (DAPT)** | Take a generally-trained model and continue pretraining it on clinical text before finetuning. | Take a professionally trained general chef and put them through an intensive, year-long molecular gastronomy specialization course. |
| **3. Finetune General Model** | Take a generally-trained model and directly finetune it on the specific, labeled clinical task. | Give a general chef a few recipes for a specific molecular gastronomy dish and ask them to perfect it. |
| **4. In-Context Learning (ICL)** | Give a massive, general-purpose model (like GPT-3) a few examples in the prompt and ask it to perform the task without any weight updates. | Hand a general chef a cookbook on molecular gastronomy and ask them to cook a complex dish on the spot, using the book only as a reference. |

### Key Finding 1: Parameter Efficiency Wins

The first major finding is that **specialized models are vastly more parameter-efficient.** A smaller model trained on in-domain clinical data can outperform a much larger generalist model.

The plot below, based on data from Table 2 in the paper, illustrates this. Notice how the 220M parameter `Clinical-T5-Base` (a specialist) outperforms the 770M parameter `T5-Large` (a generalist) on the MedNLI task.

In [ ]:
import pandas as pd

# Data from Table 2 of the paper "Do We Still Need Clinical Language Models?"
data = {
    'Model': ['T5-Base (General)', 'Clinical-T5-Base (Specialist)', 
              'T5-Large (General)', 'Clinical-T5-Large (Specialist)', 'T5-XL (General)'],
    'Parameters': ['220M', '220M', '770M', '770M', '3B'],
    'MedNLI Accuracy': [0.818, 0.855, 0.849, 0.872, 0.869],
    'Model Size (Params)': [220, 220, 770, 770, 3000] # For plotting
}
df = pd.DataFrame(data)

plt.figure(figsize=(12, 7))
colors = ['#1f77b4', '#d62728', '#1f77b4', '#d62728', '#1f77b4']
is_specialist = ['General', 'Specialist', 'General', 'Specialist', 'General']

bars = plt.bar(df['Model'], df['MedNLI Accuracy'], color=colors, alpha=0.8)
plt.ylabel('MedNLI Accuracy', fontsize=12)
plt.xlabel('Model (Size)', fontsize=12)
plt.title('Specialist Models are More Parameter-Efficient', fontsize=16, pad=20)
plt.ylim(0.8, 0.88)
plt.xticks(rotation=15, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

for bar, label in zip(bars, df['Parameters']):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.001, f'{yval:.3f}', ha='center', va='bottom')
    plt.text(bar.get_x() + bar.get_width()/2.0, 0.805, f'({label})', ha='center', va='bottom', fontsize=10, color='white')

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#d62728', alpha=0.8, label='Specialist (Clinical-Trained)'),
                   Patch(facecolor='#1f77b4', alpha=0.8, label='Generalist (Web-Trained)')]
plt.legend(handles=legend_elements, fontsize=12)

plt.tight_layout()
plt.show()

### Key Finding 2: Not All Training is Equal - In-Domain Data is More Valuable

The paper's second major finding is that performance isn't just about model size or total training compute (FLOPs); it's about the *quality* and *relevance* of the training data.

The researchers found that models trained with more **in-domain clinical tokens** consistently outperformed models trained on general web text, even if the general models were much larger or trained for more total FLOPs. This suggests that the nuanced, jargon-filled language of clinical notes provides a much richer learning signal for these tasks than generic text.

The conceptual plot below recreates Figure 3 from the paper, showing that to reach a certain performance level, clinical models are more computationally efficient.

In [ ]:
# Conceptual recreation of Figure 3 from the paper
# Log Total FLOPs vs Performance for MedNLI

# Data points are conceptual, based on the paper's findings
non_clinical_flops = np.array([46, 47, 48, 49])
non_clinical_perf = np.array([0.82, 0.85, 0.86, 0.87])

clinical_flops = np.array([48.5, 49.5])
clinical_perf = np.array([0.855, 0.872])

plt.figure(figsize=(10, 7))
plt.scatter(non_clinical_flops, non_clinical_perf, color='#1f77b4', s=100, label='Non-Clinical Models (e.g., T5)', alpha=0.8)
plt.plot(non_clinical_flops, non_clinical_perf, color='#1f77b4', linestyle='--')

plt.scatter(clinical_flops, clinical_perf, color='#d62728', s=150, marker='X', label='Clinical Models (e.g., Clinical-T5)', alpha=0.9)
plt.plot(clinical_flops, clinical_perf, color='#d62728', linestyle='--')

plt.title('Clinical Tokens Provide More Value than General Tokens', fontsize=16, pad=20)
plt.xlabel('Log Total Pretraining FLOPs (More Compute -->)', fontsize=12)
plt.ylabel('MedNLI Performance (Accuracy)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

plt.text(46.5, 0.875, 'To achieve the same performance, \nclinical models require less compute.', 
         fontsize=12, ha='center', bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.3))

plt.show()

### Key Finding 3: Finetuning Beats In-Context Learning for Clinical Tasks

Perhaps the most striking finding is the direct comparison between finetuning and In-Context Learning (ICL). The study simulated a low-data environment by finetuning models on just 1%, 5%, 10%, etc., of the available training data and compared them to massive ICL models like GPT-3.

The results were decisive: **even when finetuned on a tiny fraction of data (e.g., 1%), specialized models significantly outperformed the giant ICL models.**

This suggests that for tasks requiring deep parsing of domain-specific, structured text (like radiology reports or discharge summaries), simply showing a general model a few examples in a prompt isn't enough. The model needs to update its weights through finetuning to truly learn the domain's unique grammar and patterns.

In [ ]:
# Recreating the ablation study from Figure 4 of the paper
training_data_pct = [1, 5, 10, 25, 100]

# Simulated performance data reflecting the trends in the paper for the MedNLI task
gator_tron_perf = [0.811, 0.817, 0.837, 0.858, 0.883]
bioclin_roberta_perf = [0.824, 0.852, 0.862, 0.882, 0.900]
gpt3_icl_perf = [0.805] * 5 # ICL performance is constant regardless of training data size
flan_t5_icl_perf = [0.808] * 5

plt.figure(figsize=(12, 8))

plt.plot(training_data_pct, gator_tron_perf, marker='o', linestyle='-', label='GatorTron (Finetuned Specialist)')
plt.plot(training_data_pct, bioclin_roberta_perf, marker='s', linestyle='-', label='BioClinRoBERTa (Finetuned Specialist)')
plt.plot(training_data_pct, gpt3_icl_perf, marker='x', linestyle='--', color='red', label='GPT-3 (In-Context Learning)')
plt.plot(training_data_pct, flan_t5_icl_perf, marker='x', linestyle='--', color='purple', label='Flan-T5-XXL (In-Context Learning)')

plt.title('Finetuning Outperforms In-Context Learning, Even with Limited Data', fontsize=16, pad=20)
plt.xlabel('Percentage of Available Finetuning Data Used', fontsize=12)
plt.ylabel('MedNLI Accuracy', fontsize=12)
plt.xscale('log')
plt.xticks(training_data_pct, [f'{p}%' for p in training_data_pct])
plt.grid(True, which='both', linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

plt.annotate('Finetuned models pull ahead \n even with just 1% of data', 
             xy=(1, 0.82), xytext=(2, 0.86), 
             arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8),
             fontsize=12, bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.3))

plt.tight_layout()
plt.show()

--- 
## Part 3: A Glimpse into the Biological Frontier

The lecture also touched upon other exciting frontiers where transformers are making an impact beyond clinical text, pushing the boundaries of biology itself.

-   **Proteins (ProtNLM):** Imagine a model that can look at a long sequence of amino acids and generate a natural language description of what that protein does. This is essentially "protein captioning," and it's helping scientists characterize millions of previously unknown proteins.

-   **Genomics (Enformer & DeepConsensus):** Transformers are being used for incredibly complex genomic tasks:
    - **Error Correction (DeepConsensus):** Acting like a high-fidelity spell-checker for raw DNA sequencing data, leading to faster and more accurate genome sequencing.
    - **Gene Expression Prediction (Enformer):** Modeling how distant "enhancer" regions on DNA can loop around in 3D space to influence a gene's activity. This is crucial for understanding the genetic roots of disease.

## Conclusion: A Tale of Two Models

So, do we still need clinical language models? The answer is a resounding **yes, but it's nuanced.** There isn't one type of model to rule them all. Instead, we see a clear and complementary role for both approaches:

1.  **Generalist Giants (like Med-PaLM)** are incredibly powerful for tasks that benefit from their vast world knowledge and reasoning capabilities, such as medical question answering and simplifying complex topics for patients. Their strength lies in their breadth and their ability to be guided by techniques like instruction tuning.

2.  **Specialized Experts (like Clinical-T5)** are indispensable for tasks that require a deep, surgical understanding of the unique structure and language of clinical data. They are more parameter-efficient, cost-effective, and currently superior for deep-parsing tasks like information extraction from EHRs.

The future of biomedical AI will likely involve a powerful synergy between these two philosophies—perhaps using generalist models as a foundation, which are then efficiently specialized for specific clinical or biological tasks. This dual approach promises to unlock new discoveries, eradicate diseases, and fundamentally advance human health.